<a href="https://colab.research.google.com/github/jacomyma/mapping-controversies/blob/main/notebooks/Text_list_to_semantic_network_(embedding).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌶️ Text list to semantic network (embedding)

**Inputs:**
* a list of texts as a CSV. The texts could be short (a word or sentence) or long (articles).

**Outputs:**
* a network of the texts with semantic links obtained from an embedding and dimensionality reduction (GraphML).

## How to use
* Put your input file in the same folder as the notebook
* Edit the **settings** below if needed
* **Run all cells** (you can do that from the menu "Runtime > Run all").
  * Note: if asked to restart the runtime, do it and re-run all cells
* Scroll down to the bottom to **explore the visualization**.

### Optional section
* **RAG** (retrieval augmented generation):
  * Jump to the last section of this notebook
  * Edit the query
  * Execute that cell and those below
  * Copy-paste the generated prompt into the AI assistant of your choice

In [ ]:
# Input file
input_file = 'texts.csv'

# Which column contains the text?
text_column = 'Text'

# Limit the data points (via random sampling) to make a quick test; keep to 0 to analyze all
max_data_points = 0

# Output file
output_file = "semantic-network.graphml"

# Script starts here

In [ ]:
# Keep this to True as long as you re-run with a different input
recompute_embeddings = True

In [ ]:
# Install libraries we need
# (if already installed, it will auto-skip them, no worries)
!pip install pandas chromadb bokeh umap-learn

In [ ]:
# Imports
import pandas as pd
from sentence_transformers import SentenceTransformer
import json

In [ ]:
# Load CSV
df = pd.read_csv(input_file)
df

In [ ]:
# Shuffle row order
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Truncate data
if max_data_points>0:
  df = df.head(max_data_points)

In [ ]:
# Import necessary libraries
import chromadb
from chromadb.utils import embedding_functions

# Initialize ChromaDB and create a collection
client = chromadb.PersistentClient(path='embeddings')

if recompute_embeddings:
  # Delete the collection if it exists
  try:
      client.delete_collection(name="my_documents")
      print("Existing collection 'my_documents' deleted.")
  except:
      pass  # Ignore if collection doesn't exist

# Get or create the collection
try:
  collection = client.get_collection(name="my_documents")
except:
  collection = client.create_collection(name="my_documents")

  # Load the desired Sentence Transformer model
  embedding_model = SentenceTransformer("intfloat/multilingual-e5-large-instruct")
  # Note: Why this model?
  #       Because it is multilingual and the best with Scandinavian languages at the moment.
  #       Source: https://kennethenevoldsen.github.io/scandinavian-embedding-benchmark/

  # Get the text content and metadata from the DataFrame
  texts = df[text_column].tolist()
  metadata = df.drop(columns=[text_column]).to_dict(orient="records")

  # Use DataFrame's row index as id
  ids = df.index.astype(str).tolist()  # Convert index to strings for ChromaDB

  batch_size = 256  # Adjust this based on your resources and testing

  for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    batch_metadata = metadata[i:i + batch_size]
    batch_ids = ids[i:i + batch_size]

    batch_embeddings = embedding_model.encode(batch_texts)

    # Convert embeddings to a list of lists if necessary (depending on SentenceTransformer version)
    if not isinstance(batch_embeddings[0], list):
      batch_embeddings = [emb.tolist() for emb in batch_embeddings]

    collection.add(
      documents=batch_texts,
      metadatas=batch_metadata,
      embeddings=batch_embeddings,
      ids=batch_ids
    )
print('Done.')

In [ ]:
embeddings = collection.get(include=['embeddings'])['embeddings']

In [ ]:
umap_n_neighbors = 8

In [ ]:
# Compute UMAP for layout
import umap

if recompute_embeddings or 'umap_result' not in locals():
  # Initialize UMAP with desired parameters
  reducer = umap.UMAP(n_neighbors=umap_n_neighbors,
                      n_components=2, # dimensions
                      min_dist=0.15,
                      metric='cosine',
                      random_state=42)

  # Apply UMAP to the embeddings
  umap_result = reducer.fit_transform(embeddings)

print("UMAP reduction complete.")

In [ ]:
# Extract KNN network à la UMAP
import numpy as np
from umap import UMAP
from umap.umap_ import nearest_neighbors

def get_umap_neighbors(data, n_neighbors=15, metric='cosine'):
    # Convert data to numpy array if needed
    X = np.asarray(data)

    # Compute nearest neighbors using UMAP's internal function
    # This returns the same neighbor graph that UMAP would use
    knn_indices, knn_dists, _ = nearest_neighbors(
        X,
        n_neighbors=n_neighbors,
        metric=metric,
        metric_kwds={},
        angular=False,
        random_state=None
    )

    return knn_indices, knn_dists

indices, distances = get_umap_neighbors(embeddings, n_neighbors=umap_n_neighbors)

In [ ]:
import networkx as nx

# Create a NetworkX graph
graph = nx.Graph()

# Add nodes (ids from the dataframe)
for i in range(len(indices)):
  id = f'n{i:06d}'
  graph.add_node(id, x=umap_result[i, 0]*100, y=umap_result[i, 1]*100)
  # Add all non-text columns of df as attributes
  for col in df.columns:
    if col != text_column:
      graph.nodes[id][col] = df.iloc[i][col]

# Add edges with weights (1/distance)
for i in range(len(indices)):
  for j in range(len(indices[i])):
    neighbor_index = indices[i][j]
    distance = distances[i][j]
    if distance > 0:  # Avoid division by zero
      weight = 1 / distance
      id1 = f'n{i:06d}'
      id2 = f'n{neighbor_index:06d}'
      graph.add_edge(id1, id2, weight=weight)

# Save the graph as a GEXF file
nx.write_graphml(graph, output_file)

In [ ]:
import bokeh
import bokeh.plotting as bp
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper

In [ ]:
# Create a ColumnDataSource for Bokeh
source = ColumnDataSource(data=dict(
    x=umap_result[:, 0],
    y=umap_result[:, 1],
    text=df[text_column],
    id=df.index.astype(str),
))

# Exploratory interface below
It will appear once the cell is executed successfully.

In [ ]:
# Output the plot to the notebook
output_notebook()

# Create the figure
p = bp.figure(width=900, height=900,
            title=f"Semantic map (proximity indicates similarity)",
            tools="pan,wheel_zoom,box_zoom,reset,hover", tooltips=[("Text", "@text")], match_aspect=True)

# Add scatter plot with color mapping and hover tool
p.scatter('x', 'y', source=source, size=10)

# Customize the plot (optional)
p.xaxis.axis_label = "UMAP Dimension 1"
p.yaxis.axis_label = "UMAP Dimension 2"

# HTML tooltip
hover = p.select(dict(type=HoverTool))
hover.tooltips = """
    <div style="width: 300px; word-wrap: break-word;">
        <div>@text</div>
        <div><em>&mdash;@actor, @source, @date</em></div>
        <div>@issue</div>
        <pre>ID: @id</pre>
        <br>
    </div>
"""
hover.mode = 'mouse'  # Enable HTML rendering

# Show the plot
show(p)

# Retrieval Augmented Generation (RAG)
Edit the cell below (query), run it and the subsequent cells, then copy the last output into ChatGPT or another AI assistant.

In [ ]:
# EDIT THE QUERY BELOW
# Then execute this cell and the following ones

query = "What is energy conversion?"
number_of_results = 10

In [ ]:
# Embed the query
query_embedding = embedding_model.encode([query])

# Perform a similarity search
results = collection.query(
    query_embeddings=query_embedding,
    n_results=number_of_results # Number of chunks retrieved
)

# Extract the chunks from results
retrievedChunks_txt = results['documents'][0]
retrievedChunks_id = results['ids'][0]

# Print them
print("# Extracted sentences:\n")
for chunk in retrievedChunks_txt:
  print("\n- "+chunk)

In [ ]:
# Plot on the Bokeh figure

# Create the figure
p = bp.figure(width=900, height=900,
            title="UMAP Visualization of text chunks (colored by source document)",
            tools="pan,wheel_zoom,box_zoom,reset,hover", tooltips=[("ID", "@id"), ("Text", "@text")], match_aspect=True)

# Add scatter plot with color mapping and hover tool
p.scatter('x', 'y', source=source, size=10)

# Highlight closest chunks in red
closest_indices = df[df.index.astype(str).isin(retrievedChunks_id)].index

closest_source = ColumnDataSource(data=dict(
    x=[umap_result[i, 0] for i in closest_indices],
    y=[umap_result[i, 1] for i in closest_indices],
))

p.circle('x', 'y', source=closest_source, size=15, color="red", legend_label="Closest Chunks")

# Customize the plot (optional)
p.xaxis.axis_label = "UMAP Dimension 1"
p.yaxis.axis_label = "UMAP Dimension 2"

# HTML tooltip
hover = p.select(dict(type=HoverTool))
hover.tooltips = """
    <div style="width: 300px; word-wrap: break-word;">
        <div>@text</div>
        <div><em>&mdash;@actor, @source, @date</em></div>
        <div>@issue</div>
        <pre>ID: @id</pre>
        <br>
    </div>
"""
hover.mode = 'mouse'  # Enable HTML rendering

# Show the plot
show(p)

In [ ]:
prompt = f'''You are an assistant answering queries informed by a specific context provided below in JSON.

Given the context information and not prior knowledge, answer the query.

QUERY
```
{query}
```

CONTEXT
```json
{json.dumps(retrievedChunks_txt)}
```
'''

print(prompt)